# Data Acquisition and Cleaning
For this part, we will mainly use
<br>The Oscar Award, 1927 - 2020 from kaggle https://www.kaggle.com/unanimad/the-oscar-award as a starting point
<br>TMDB API
<br>OMDB API

In [1]:
# Importing neccesary libraries 
from tmdbv3api import TMDb, Movie, Discover, Genre
from tmdbv3api import Account
from tmdbv3api import Authentication
import numpy as np
import random
import pandas as pd
from collections import defaultdict
import requests as rq
import re
import traceback
import datetime
from collections import Counter
import omdb

In [2]:
##TMDB details
##Account details for getting movie reccomendations
##Run this block only once during a session
tmdb = TMDb()
tmdb.api_key = '0921b0cce35c0b2ec8b874614d1d0b47'
tmdb.language = 'en'

USERNAME = "cybercat94"
PASSWORD = "Wave1994@"
auth = Authentication(username=USERNAME, password=PASSWORD)
account = Account()
details = account.details()

print("You are logged in as %s. Your account ID is %s." % (details.username, details.id))
print("This session expires at: %s" % auth.expires_at)

You are logged in as cybercat94. Your account ID is 10198725.
This session expires at: 2021-03-25 05:16:44 UTC


## Scrapping and cleaning oscar_award_data.csv
We will start by scrapping the dataset to meet our needs and requirements.

In [3]:
oscar_df = pd.read_csv('data/the_oscar_award.csv')
oscar_df

,year_film,year_ceremony,ceremony,category,name,film,winner
0,1927,1928,1,ACTOR,Richard Barthelmess,The Noose,False
1,1927,1928,1,ACTOR,Emil Jannings,The Last Command,True
2,1927,1928,1,ACTRESS,Louise Dresser,A Ship Comes In,False
3,1927,1928,1,ACTRESS,Janet Gaynor,7th Heaven,True
4,1927,1928,1,ACTRESS,Gloria Swanson,Sadie Thompson,False
...,...,...,...,...,...,...,...
10390,2019,2020,92,WRITING (Original Screenplay),"Screenplay by Bong Joon Ho, Han Jin Won; Story...",Parasite,True
10391,2019,2020,92,JEAN HERSHOLT HUMANITARIAN AWARD,Geena Davis,NaN,True
10392,2019,2020,92,HONORARY AWARD,David Lynch,NaN,True
10393,2019,2020,92,HONORARY AWARD,Wes Studi,NaN,True


Renaming columns for better readabality and querying data from the year 1960 to 2020.

In [4]:
oscar_df= oscar_df.rename({'category':'oscar_category', 'name':'nominee', 'winner':'oscar_win'}, axis=1)
oscar_df.drop(['ceremony'], axis=1, inplace=True)
oscar_df = oscar_df.query('year_film >=1960')

Next is to get the total counts of nominations for each film.

In [5]:
no_of_nomination_df = oscar_df.groupby('film').size().reset_index(name='no_oscar_nominations')

For this project we will be focusing on major awards only, the so called "big 5".
<br>Best Actor
<br>Best Actress
<br>Best Director
<br>Best Picture
<br>Best Adapted Screenplay/Best Original Screenplay

<br>As the title of these awards kept changing through the years, we will have to filter through the data.


In [6]:
oscar_df = oscar_df.replace(to_replace={'oscar_category' : {'ACTOR IN A LEADING ROLE': 'Best Actor',
                                                            'ACTOR': 'Best Actor',
                                                            'ACTRESS': 'Best Actress',
                                                            'ACTRESS IN A LEADING ROLE': 'Best Actress',
                                                            'DIRECTING' : 'Best Director',
                                                            'BEST PICTURE' : 'Best Picture',
                                                            'BEST MOTION PICTURE' : 'Best Picture',
                                                            'WRITING (Adapted Screenplay)' : 'Best Adapted Screenplay',
                                                            'WRITING (Screenplay Adapted from Other Material)' : 'Best Adapted Screenplay',
                                                            'WRITING (Screenplay Based on Material from Another Medium)' : 'Best Adapted Screenplay',
                                                            'WRITING (Screenplay--based on material from another medium)' : 'Best Adapted Screenplay',
                                                            'WRITING (Screenplay Based on Material Previously Produced or Published)' : 'Best Adapted Screenplay',
                                                            'WRITING (Original Screenplay)' : 'Best Original Screenplay',
                                                            'WRITING (Screenplay Written Directly for the Screen)' : 'Best Original Screenplay',
                                                            'WRITING (Story and Screenplay--based on factual material or material not previously published or produced)' : 'Best Original Screenplay',
                                                            'WRITING (Story and Screenplay--written directly for the screen)' : 'Best Original Screenplay',
                                                            'WRITING (Screenplay Written Directly for the Screen--based on factual material or on story material not previously published or produced)' : 'Best Original Screenplay',
                                                            'WRITING (Story and Screenplay--based on material not previously published or produced)' : 'Best Original Screenplay'},}, value=None)

oscar_df = oscar_df.query('oscar_category in ["Best Actor", "Best Actress", "Best Director", "Best Picture", "Best Adapted Screenplay","Best Original Screenplay"]')
oscar_df = pd.merge(oscar_df, no_of_nomination_df, on=['film'], how='left')
oscar_df = oscar_df.reset_index()
oscar_df.drop(['index'], axis=1, inplace=True)
oscar_df

,year_film,year_ceremony,oscar_category,nominee,film,oscar_win,no_oscar_nominations
0,1960,1961,Best Actor,Trevor Howard,Sons and Lovers,False,7
1,1960,1961,Best Actor,Burt Lancaster,Elmer Gantry,True,5
2,1960,1961,Best Actor,Jack Lemmon,The Apartment,False,10
3,1960,1961,Best Actor,Laurence Olivier,The Entertainer,False,1
4,1960,1961,Best Actor,Spencer Tracy,Inherit the Wind,False,4
...,...,...,...,...,...,...,...
1835,2019,2020,Best Original Screenplay,Written by Rian Johnson,Knives Out,False,1
1836,2019,2020,Best Original Screenplay,Written by Noah Baumbach,Marriage Story,False,6
1837,2019,2020,Best Original Screenplay,Written by Sam Mendes & Krysty Wilson-Cairns,1917,False,10
1838,2019,2020,Best Original Screenplay,Written by Quentin Tarantino,Once upon a Time...in Hollywood,False,10


In [21]:
df = df.groupby('film')['oscar_category'].nunique()

print (df)

AttributeError: 'int' object has no attribute 'groupby'

In [7]:
db_path='FullDB_''.csv'
oscar_df.to_csv(db_path)
print("Exported to local folder:",db_path)

Exported to local folder: FullDB_.csv


## Get Movie Information from TMDB api
This function uses TMDB api(DISCOVER), to retrieve all the movies with the constraint year.
<br> It will retrieve basic details such as id, title, list of genre ids, vote average and lastly vote count.
<br> We will later retrieve more information using the getDetails function to get details like casts, crews and etc.

In [8]:
def get_movies_by_year(year):

    movie = Movie()

    # initialiZe empty dictionary
    _movie_dict = {"tmdb_id":[], 
                   "year":[], 
                   "title":[], 
                   "genre":[], 
                   "tmdb_vote_average":[], 
                   "tmdb_vote_count":[]};
    
    print(f"In {year}")
    
    for i in range(1,501):
        discover = Discover()
        movie = discover.discover_movies({
            'year': year,
            'page': i,
            'certification_country': 'US'
        })
        
        for m in movie:
            _movie_dict["tmdb_id"].append(m.id)
            _movie_dict["year"].append(year)
            _movie_dict["title"].append(m.title)
            _movie_dict["genre"].append(m.title)
            _movie_dict["tmdb_vote_average"].append(m.title)
            _movie_dict["tmdb_vote_count"].append(m.title)

        if i == int(discover.total_pages):
            break

    return _movie_dict

function to translate the genre id

In [9]:
def translate_genre_id(g):
    for i in range(0,len(g)):
        if g[i] == 28:
            g[i] = 'Action'
        elif g[i] == 12:
            g[i] = 'Adventure'
        elif g[i] == 16:
            g[i] = 'Animation'
        elif g[i] == 35:
            g[i] = 'Comedy'
        elif g[i] == 80:
            g[i] = 'Crime'
        elif g[i] == 99:
            g[i] = 'Documentary'
        elif g[i] == 18:
            g[i] = 'Drama'
        elif g[i] == 10751:
            g[i] = 'Family'
        elif g[i] == 14:
            g[i] = 'Fantasy'
        elif g[i] == 36:
            g[i] = 'History'
        elif g[i] == 27:
            g[i] = 'Horror'
        elif g[i] == 10402:
            g[i] = 'Music'
        elif g[i] == 9648:
            g[i] = 'Mystery'
        elif g[i] == 10749:
            g[i] = 'Romance'
        elif g[i] == 878:
            g[i] = 'Science Fiction'
        elif g[i] == 10770:
            g[i] = 'TV Movie'
        elif g[i] == 53:
            g[i] = 'Thriller'
        elif g[i] == 10752:
            g[i] = 'War'
        elif g[i] == 37:
            g[i] = 'Western'
    return g

Function to retrieve movie details with movie name and year

In [24]:
def search_movie_by_name_year(details_dict):
    
    oscar_dict = {"tmdb_id":[], 
                  "year_film":[], 
                  "year_ceremony":[], 
                  "film":[], 
                  "oscar_category":[], 
                  "nominee":[],
                  "no_oscar_nominations":[], 
                  "oscar_win":[],
                  "genre_id":[], 
                  "tmdb_vote_average":[],
                  "tmdb_vote_count":[]};
    
    movie = Movie()
    for item in details_dict:
        s = movie.search(item["film"])
        size = len(s)
        for i in range(0, size):
            result = s[i]
            print(item["film"])
            print(item["oscar_category"])
            try:
                if (result.release_date[0:4] == str(item["year_film"]) or result.release_date[0:4] == str(item["year_film"]-1) or result.release_date[0:4] == str(item["year_film"]-2)):
                    #print("found movie")
                    oscar_dict["tmdb_id"].append(result.id)
                    oscar_dict["year_film"].append(item["year_film"])
                    oscar_dict["year_ceremony"].append(item["year_ceremony"])
                    oscar_dict["oscar_category"].append(item["oscar_category"])
                    oscar_dict["film"].append(item["film"])
                    oscar_dict["nominee"].append(item["nominee"])
                    oscar_dict["no_oscar_nominations"].append(item["no_oscar_nominations"])
                    oscar_dict["oscar_win"].append(item["oscar_win"])
                    oscar_dict["genre_id"].append(translate_genre_id(result.genre_ids))
                    oscar_dict["tmdb_vote_average"].append(result.vote_average)
                    oscar_dict["tmdb_vote_count"].append(result.vote_count)
                    break
            except NameError:
                print("no release date")
            except:
                print("Error can't find movie")

    return oscar_dict

In [25]:
details_dict = oscar_df.to_dict('r')
df = pd.DataFrame(search_movie_by_name_year(details_dict))

Sons and Lovers
Best Actor
Elmer Gantry
Best Actor
The Apartment
Best Actor
The Entertainer
Best Actor
Inherit the Wind
Best Actor
Sunrise at Campobello
Best Actress
The Sundowners
Best Actress
The Apartment
Best Actress
Never on Sunday
Best Actress
Never on Sunday
Best Actress
Butterfield 8
Best Actress
The Apartment
Best Director
Never on Sunday
Best Director
Never on Sunday
Best Director
Psycho
Best Director
Psycho
Best Director
Sons and Lovers
Best Director
The Sundowners
Best Director
The Alamo
Best Picture
The Alamo
Best Picture
The Apartment
Best Picture
Elmer Gantry
Best Picture
Sons and Lovers
Best Picture
The Sundowners
Best Picture
Elmer Gantry
Best Adapted Screenplay
Inherit the Wind
Best Adapted Screenplay
Sons and Lovers
Best Adapted Screenplay
The Sundowners
Best Adapted Screenplay
Tunes of Glory
Best Adapted Screenplay
The Angry Silence
Best Original Screenplay
The Apartment
Best Original Screenplay
The Facts of Life
Best Original Screenplay
Never on Sunday
Best Origina

Face to Face
Best Director
Face to Face
Best Director
Face to Face
Best Director
Face to Face
Best Director
Face to Face
Best Director
Face to Face
Best Director
Face to Face
Best Director
Face to Face
Best Director
Face to Face
Best Director
Face to Face
Best Director
Face to Face
Best Director
Face to Face
Best Director
Face to Face
Best Director
Face to Face
Best Director
Face to Face
Best Director
Face to Face
Best Director
Face to Face
Best Director
Face to Face
Best Director
Face to Face
Best Director
Face to Face
Best Director
Network
Best Director
Network
Best Director
Rocky
Best Director
Seven Beauties
Best Director
All the President's Men
Best Picture
Bound for Glory
Best Picture
Network
Best Picture
Network
Best Picture
Rocky
Best Picture
Taxi Driver
Best Picture
All the President's Men
Best Adapted Screenplay
Bound for Glory
Best Adapted Screenplay
Fellini's Casanova
Best Adapted Screenplay
The Seven-per-cent Solution
Best Adapted Screenplay
Voyage of the Damned
Best Adapte

Bugsy
Best Actor
Cape Fear
Best Actor
The Silence of the Lambs
Best Actor
The Prince of Tides
Best Actor
The Fisher King
Best Actor
Thelma & Louise
Best Actress
Rambling Rose
Best Actress
The Silence of the Lambs
Best Actress
For the Boys
Best Actress
Thelma & Louise
Best Actress
Boyz N the Hood
Best Director
Bugsy
Best Director
JFK
Best Director
The Silence of the Lambs
Best Director
Thelma & Louise
Best Director
Beauty and the Beast
Best Picture
Bugsy
Best Picture
JFK
Best Picture
The Prince of Tides
Best Picture
The Silence of the Lambs
Best Picture
Europa Europa
Best Adapted Screenplay
Fried Green Tomatoes
Best Adapted Screenplay
JFK
Best Adapted Screenplay
The Prince of Tides
Best Adapted Screenplay
The Silence of the Lambs
Best Adapted Screenplay
Boyz N the Hood
Best Original Screenplay
Bugsy
Best Original Screenplay
The Fisher King
Best Original Screenplay
Grand Canyon
Best Original Screenplay
Thelma & Louise
Best Original Screenplay
Chaplin
Best Actor
Unforgiven
Best Actor
Scen

Ray
Best Director
Sideways
Best Director
Vera Drake
Best Director
The Aviator
Best Picture
Finding Neverland
Best Picture
Million Dollar Baby
Best Picture
Ray
Best Picture
Sideways
Best Picture
Before Sunset
Best Adapted Screenplay
Finding Neverland
Best Adapted Screenplay
Million Dollar Baby
Best Adapted Screenplay
The Motorcycle Diaries
Best Adapted Screenplay
Sideways
Best Adapted Screenplay
The Aviator
Best Original Screenplay
Eternal Sunshine of the Spotless Mind
Best Original Screenplay
Hotel Rwanda
Best Original Screenplay
The Incredibles
Best Original Screenplay
Vera Drake
Best Original Screenplay
Capote
Best Actor
Hustle & Flow
Best Actor
Brokeback Mountain
Best Actor
Walk the Line
Best Actor
Good Night, and Good Luck.
Best Actor
Mrs. Henderson Presents
Best Actress
Transamerica
Best Actress
Pride & Prejudice
Best Actress
North Country
Best Actress
Walk the Line
Best Actress
Brokeback Mountain
Best Director
Capote
Best Director
Crash
Best Director
Crash
Best Director
Good Nigh

Vice
Best Original Screenplay
Vice
Best Original Screenplay
Pain and Glory
Best Actor
Once upon a Time...in Hollywood
Best Actor
Marriage Story
Best Actor
Joker
Best Actor
The Two Popes
Best Actor
Harriet
Best Actress
Marriage Story
Best Actress
Little Women
Best Actress
Bombshell
Best Actress
Judy
Best Actress
The Irishman
Best Director
Joker
Best Director
1917
Best Director
Once upon a Time...in Hollywood
Best Director
Parasite
Best Director
Parasite
Best Director
Parasite
Best Director
Ford v Ferrari
Best Picture
The Irishman
Best Picture
Jojo Rabbit
Best Picture
Joker
Best Picture
Little Women
Best Picture
Marriage Story
Best Picture
1917
Best Picture
Once upon a Time...in Hollywood
Best Picture
Parasite
Best Picture
Parasite
Best Picture
Parasite
Best Picture
The Irishman
Best Adapted Screenplay
Jojo Rabbit
Best Adapted Screenplay
Joker
Best Adapted Screenplay
Little Women
Best Adapted Screenplay
The Two Popes
Best Adapted Screenplay
Knives Out
Best Original Screenplay
Marriage St

In [17]:
df

,tmdb_id,year_film,year_ceremony,film,oscar_category,nominee,no_oscar_nominations,oscar_win,genre_id,tmdb_vote_average,tmdb_vote_count
0,53939,1960,1961,Sons and Lovers,Best Actor,Trevor Howard,7,False,[Drama],6.9,18
1,22013,1960,1961,Elmer Gantry,Best Actor,Burt Lancaster,5,True,[Drama],7.2,85
2,284,1960,1961,The Apartment,Best Actor,Jack Lemmon,10,False,"[Comedy, Drama, Romance]",8.2,1425
3,18929,1960,1961,The Entertainer,Best Actor,Laurence Olivier,1,False,"[Drama, Music]",6.2,28
4,1908,1960,1961,Inherit the Wind,Best Actor,Spencer Tracy,4,False,"[Drama, History]",7.8,250
...,...,...,...,...,...,...,...,...,...,...,...
1833,546554,2019,2020,Knives Out,Best Original Screenplay,Written by Rian Johnson,1,False,"[Comedy, Crime, Drama, Mystery]",7.8,6929
1834,492188,2019,2020,Marriage Story,Best Original Screenplay,Written by Noah Baumbach,6,False,[Drama],7.8,4695
1835,530915,2019,2020,1917,Best Original Screenplay,Written by Sam Mendes & Krysty Wilson-Cairns,10,False,"[War, Drama, Action, Thriller]",7.9,7862
1836,466272,2019,2020,Once upon a Time...in Hollywood,Best Original Screenplay,Written by Quentin Tarantino,10,False,"[Comedy, Drama, Thriller]",7.5,8386


Function to retrieve top 4 casts 

In [26]:
def get_top_4_casts(creds):
    
    cast = (creds["cast"])
    cast_list = []
    
    for i in range(0,4):
        try:
            cast_list.append(cast[i].name)
        except:
            print("OUT OF RANGE")
        
    return cast_list

Function to retrieve total popularity score of top 4 casts 

In [27]:
def get_top_4_casts_popularity_score(creds):
    
    cast = (creds["cast"])
    popularity_score = 0

    for i in range(0,4):
        try:
            popularity_score += cast[i].popularity
        except:
            print("OUT OF RANGE")
        
    return popularity_score

Function to retrieve crew based on positions

In [28]:
def get_crew_position(creds, job_id):
    crew = (creds["crew"])
    producer_list = []
    screenplay_list = []
    for c in crew:
        try:
            if c.job == 'Director' and job_id == 0:
                director = c.name
            elif c.job == 'Producer'and job_id == 1:
                producer_list.append(c.name)
            elif c.job == 'Screenplay'and job_id == 2:
                screenplay_list.append(c.name)
        except:
             print("OUT OF RANGE")
    if job_id == 0:
        return director
    elif job_id == 1:
        return producer_list
    elif job_id == 2:
        return screenplay_list

Function to get more movie details

In [29]:
def get_movie_details(oscar_dict):
    
    movie = Movie()
    
    movie_detials_dict = {"tmdb_id":[],
                           "imdb_id":[],
                           "year_film":[],
                           "year_ceremony":[], 
                           "film":[], 
                           "oscar_category":[], 
                           "nominee":[],
                           "no_oscar_nominations":[], 
                           "oscar_win":[],
                           "genre_id":[],
                           "staring_casts":[],
                           "casts_popularity":[],
                           "director":[],
                           "producers":[],
                           "screenplay":[],
                           "budget":[],
                           "revenue":[],
                           "runtime":[],
                           "tmdb_vote_average":[], 
                           "tmdb_vote_count":[]};
                        
    for item in oscar_dict:
        if item["tmdb_id"] != 0:
            m = movie.details(item["tmdb_id"])
            movie_detials_dict["tmdb_id"].append(item["tmdb_id"])
            movie_detials_dict["imdb_id"].append(m.imdb_id)
            movie_detials_dict["year_film"].append(item["year_film"])
            movie_detials_dict["year_ceremony"].append(item["year_ceremony"])
            movie_detials_dict["film"].append(item["film"])
            movie_detials_dict["oscar_category"].append(item["oscar_category"])
            movie_detials_dict["nominee"].append(item["nominee"])
            movie_detials_dict["no_oscar_nominations"].append(item["no_oscar_nominations"])
            movie_detials_dict["oscar_win"].append(item["oscar_win"])
            movie_detials_dict["genre_id"].append(item["genre_id"])
            movie_detials_dict["staring_casts"].append(get_top_4_casts(m.casts))
            movie_detials_dict["casts_popularity"].append(get_top_4_casts_popularity_score(m.casts))
            movie_detials_dict["director"].append(get_crew_position(m.casts, 0))
            movie_detials_dict["producers"].append(get_crew_position(m.casts, 1))
            movie_detials_dict["screenplay"].append(get_crew_position(m.casts, 2))
            movie_detials_dict["budget"].append(m.budget)
            movie_detials_dict["revenue"].append(m.revenue)
            movie_detials_dict["runtime"].append(m.runtime)
            movie_detials_dict["tmdb_vote_average"].append(item["tmdb_vote_average"])
            movie_detials_dict["tmdb_vote_count"].append(item["tmdb_vote_count"])
            print("Success")
                          
    return movie_detials_dict


In [30]:
oscar_dict = df.to_dict('r')
final_df = pd.DataFrame(get_movie_details(oscar_dict))

Success
Success
Success
Success
Success
Success
Success
Success
Success
Success
Success
Success
Success
Success
Success
Success
Success
Success
Success
Success
Success
Success
Success
Success
Success
Success
Success
Success
Success
Success
Success
Success
Success
Success
Success
Success
Success
Success
Success
Success
Success
Success
Success
Success
Success
Success
Success
Success
Success
Success
Success
Success
Success
Success
Success
Success
Success
Success
Success
Success
Success
Success
Success
Success
Success
Success
Success
Success
Success
Success
Success
Success
Success
Success
Success
Success
Success
Success
Success
Success
Success
Success
Success
Success
Success
Success
Success
Success
Success
Success
Success
Success
Success
Success
Success
Success
Success
Success
Success
Success
Success
Success
Success
Success
Success
Success
Success
Success
Success
Success
Success
Success
Success
Success
Success
Success
Success
Success
Success
Success
Success
Success
Success
Success
Success


Success
Success
Success
Success
Success
Success
Success
Success
Success
Success
Success
Success
Success
Success
Success
Success
Success
Success
Success
Success
Success
Success
Success
Success
Success
Success
Success
Success
Success
Success
Success
Success
Success
Success
Success
Success
Success
Success
Success
Success
Success
Success
Success
Success
Success
Success
Success
Success
Success
Success
Success
Success
Success
Success
Success
Success
Success
Success
Success
Success
Success
Success
Success
Success
Success
Success
Success
Success
Success
Success
Success
Success
Success
Success
Success
Success
Success
Success
Success
Success
Success
Success
Success
Success
Success
Success
Success
Success
Success
Success
Success
Success
Success
Success
Success
Success
Success
Success
Success
Success
Success
Success
Success
Success
Success
Success
Success
Success
Success
Success
Success
Success
Success
Success
Success
Success
Success
Success
Success
Success
Success
Success
Success
Success
Success


In [31]:
final_df

,tmdb_id,imdb_id,year_film,year_ceremony,film,oscar_category,nominee,no_oscar_nominations,oscar_win,genre_id,staring_casts,casts_popularity,director,producers,screenplay,budget,revenue,runtime,tmdb_vote_average,tmdb_vote_count
0,53939,tt0054326,1960,1961,Sons and Lovers,Best Actor,Trevor Howard,7,False,[Drama],"[Trevor Howard, Dean Stockwell, Wendy Hiller, ...",13.237,Jack Cardiff,[],[Gavin Lambert],0,0,103,6.9,18
1,22013,tt0053793,1960,1961,Elmer Gantry,Best Actor,Burt Lancaster,5,True,[Drama],"[Burt Lancaster, Jean Simmons, Arthur Kennedy,...",8.478,Richard Brooks,[Bernard Smith],[Richard Brooks],3000000,10400000,146,7.2,85
2,284,tt0053604,1960,1961,The Apartment,Best Actor,Jack Lemmon,10,False,"[Comedy, Drama, Romance]","[Jack Lemmon, Shirley MacLaine, Fred MacMurray...",16.337,Billy Wilder,[Billy Wilder],[],3000000,25000000,125,8.2,1425
3,18929,tt0053796,1960,1961,The Entertainer,Best Actor,Laurence Olivier,1,False,"[Drama, Music]","[Laurence Olivier, Brenda De Banzie, Roger Liv...",6.567,Tony Richardson,[Harry Saltzman],"[John Osborne, Nigel Kneale]",0,0,96,6.2,28
4,1908,tt0053946,1960,1961,Inherit the Wind,Best Actor,Spencer Tracy,4,False,"[Drama, History]","[Spencer Tracy, Fredric March, Gene Kelly, Dic...",9.098,Stanley Kramer,"[Stanley Kramer, Herman Shumlin]","[Nedrick Young, Harold Jacob Smith]",0,0,128,7.8,250
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1833,546554,tt8946378,2019,2020,Knives Out,Best Original Screenplay,Written by Rian Johnson,1,False,"[Comedy, Crime, Drama, Mystery]","[Daniel Craig, Chris Evans, Ana de Armas, Jami...",60.512,Rian Johnson,"[Rian Johnson, Ram Bergman]",[],40000000,309232797,131,7.8,6929
1834,492188,tt7653254,2019,2020,Marriage Story,Best Original Screenplay,Written by Noah Baumbach,6,False,[Drama],"[Adam Driver, Scarlett Johansson, Laura Dern, ...",57.094,Noah Baumbach,"[Noah Baumbach, David Heyman]",[],19000000,2300000,137,7.8,4695
1835,530915,tt8579674,2019,2020,1917,Best Original Screenplay,Written by Sam Mendes & Krysty Wilson-Cairns,10,False,"[War, Drama, Action, Thriller]","[George MacKay, Dean-Charles Chapman, Mark Str...",21.306,Sam Mendes,"[Sam Mendes, Callum McDougall, Pippa Harris, J...","[Sam Mendes, Krysty Wilson-Cairns]",100000000,374733942,119,7.9,7862
1836,466272,tt7131622,2019,2020,Once upon a Time...in Hollywood,Best Original Screenplay,Written by Quentin Tarantino,10,False,"[Comedy, Drama, Thriller]","[Leonardo DiCaprio, Brad Pitt, Margot Robbie, ...",65.053,Quentin Tarantino,"[Quentin Tarantino, David Heyman, Shannon McIn...",[Quentin Tarantino],95000000,374251247,162,7.5,8386


In [32]:
oscar_details = pd.merge(final_df, oscar_df, how='right')
oscar_details

,tmdb_id,imdb_id,year_film,year_ceremony,film,oscar_category,nominee,no_oscar_nominations,oscar_win,genre_id,staring_casts,casts_popularity,director,producers,screenplay,budget,revenue,runtime,tmdb_vote_average,tmdb_vote_count
0,53939.0,tt0054326,1960,1961,Sons and Lovers,Best Actor,Trevor Howard,7,False,[Drama],"[Trevor Howard, Dean Stockwell, Wendy Hiller, ...",13.237,Jack Cardiff,[],[Gavin Lambert],0.0,0.0,103.0,6.9,18.0
1,22013.0,tt0053793,1960,1961,Elmer Gantry,Best Actor,Burt Lancaster,5,True,[Drama],"[Burt Lancaster, Jean Simmons, Arthur Kennedy,...",8.478,Richard Brooks,[Bernard Smith],[Richard Brooks],3000000.0,10400000.0,146.0,7.2,85.0
2,284.0,tt0053604,1960,1961,The Apartment,Best Actor,Jack Lemmon,10,False,"[Comedy, Drama, Romance]","[Jack Lemmon, Shirley MacLaine, Fred MacMurray...",16.337,Billy Wilder,[Billy Wilder],[],3000000.0,25000000.0,125.0,8.2,1425.0
3,18929.0,tt0053796,1960,1961,The Entertainer,Best Actor,Laurence Olivier,1,False,"[Drama, Music]","[Laurence Olivier, Brenda De Banzie, Roger Liv...",6.567,Tony Richardson,[Harry Saltzman],"[John Osborne, Nigel Kneale]",0.0,0.0,96.0,6.2,28.0
4,1908.0,tt0053946,1960,1961,Inherit the Wind,Best Actor,Spencer Tracy,4,False,"[Drama, History]","[Spencer Tracy, Fredric March, Gene Kelly, Dic...",9.098,Stanley Kramer,"[Stanley Kramer, Herman Shumlin]","[Nedrick Young, Harold Jacob Smith]",0.0,0.0,128.0,7.8,250.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1835,546554.0,tt8946378,2019,2020,Knives Out,Best Original Screenplay,Written by Rian Johnson,1,False,"[Comedy, Crime, Drama, Mystery]","[Daniel Craig, Chris Evans, Ana de Armas, Jami...",60.512,Rian Johnson,"[Rian Johnson, Ram Bergman]",[],40000000.0,309232797.0,131.0,7.8,6929.0
1836,492188.0,tt7653254,2019,2020,Marriage Story,Best Original Screenplay,Written by Noah Baumbach,6,False,[Drama],"[Adam Driver, Scarlett Johansson, Laura Dern, ...",57.094,Noah Baumbach,"[Noah Baumbach, David Heyman]",[],19000000.0,2300000.0,137.0,7.8,4695.0
1837,530915.0,tt8579674,2019,2020,1917,Best Original Screenplay,Written by Sam Mendes & Krysty Wilson-Cairns,10,False,"[War, Drama, Action, Thriller]","[George MacKay, Dean-Charles Chapman, Mark Str...",21.306,Sam Mendes,"[Sam Mendes, Callum McDougall, Pippa Harris, J...","[Sam Mendes, Krysty Wilson-Cairns]",100000000.0,374733942.0,119.0,7.9,7862.0
1838,466272.0,tt7131622,2019,2020,Once upon a Time...in Hollywood,Best Original Screenplay,Written by Quentin Tarantino,10,False,"[Comedy, Drama, Thriller]","[Leonardo DiCaprio, Brad Pitt, Margot Robbie, ...",65.053,Quentin Tarantino,"[Quentin Tarantino, David Heyman, Shannon McIn...",[Quentin Tarantino],95000000.0,374251247.0,162.0,7.5,8386.0


In [37]:
db_path='check1''.csv'
oscar_details.to_csv(db_path)
print("Exported to local folder:",db_path)

Exported to local folder: check1.csv


In [36]:
omdb.set_default('apikey', '4a7f4869')
omdb.set_default('tomatoes', True)

In [ ]:
omdb.imdbid('tt6751668', tomatoes = True)

In [ ]:
print(omdb.request(t='Parasite', y=2019, plot='full', tomatoes='true', timeout=5))

In [ ]:
movie = Movie()

s = movie.search("The Damned")
print(len(s))